# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [2]:
input_keyword = "互斥鎖"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

互斥鎖: %E4%BA%92%E6%96%A5%E9%8E%96
/wiki/%E4%BA%92%E6%96%A5%E9%8E%96


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


互斥鎖（英語：Mutual exclusion，縮寫 Mutex）是一種用於多執行緒編程中，防止兩條執行緒同時對同一公共資源（比如全域變數）進行讀寫的機制。該目的通過將代碼切片成一個一個的臨界區域（critical section）達成。臨界區域指的是一塊對公共資源進行存取的代碼，並非一種機制或是演算法。一個程式、行程、執行緒可以擁有多個臨界區域，但是並不一定會應用互斥鎖。

需要此機制的資源的例子有：旗標、佇列、計數器、中斷處理程式等用於在多條並列執行的代碼間傳遞資料、同步狀態等的資源。維護這些資源的同步、一致和完整是很困難的，因為一條執行緒可能在任何一個時刻被暫停（休眠）或者恢復（喚醒）。

例如：一段代碼（甲）正在分步修改一塊資料。這時，另一條執行緒（乙）由於一些原因被喚醒。如果乙此時去讀取甲正在修改的資料，而甲碰巧還沒有完成整個修改過程，這個時候這塊資料的狀態就處在極大的不確定狀態中，讀取到的資料當然也是有問題的。更嚴重的情況是乙也往這塊地方寫資料，這樣的一來，後果將變得不可收拾。因此，多個執行緒間共享的資料必須被保護。達到這個目的的方法，就是確保同一時間只有一個臨界區域處於執行狀態，而其他的臨界區域，無論是讀是寫，都必須被掛起並且不能獲得執行機會。

依實現方式可分為硬體實現和軟體實現兩種。

單核心系統上最常見的方式就是關閉儘可能多的可能對共享資料段進行讀寫的指令中斷。這樣一來就可以避免在臨界區域中暫停程式執行，或是來自硬體的要求修改目標共享資料段的中斷請求。多核心系統上則通過檢查並置位（取得原始值並指定新值）機制達成，當一個核心需要另一個核心占用的資源的時候，該核心將不斷的查詢所有核心間共享的占用旗標，直到另一個核心將占用旗標復位為未使用為止。相關的虛擬碼如下所示：

lock的值為1則表示鎖被占用，為0則是空閒。

在檢查並置位機制中，一個核心在對旗標執行讀寫的過程當中不會釋放占用的存取匯流排。該種方法又稱為自旋鎖。

類似的原子操作，如比較並交換機制，則更常用於對連結串列等資料結構進行不阻斷同步。

類似的方式也有通過軟體類比達成的。但是該種類比會對電腦造成極大的負荷，因為申請占用自旋鎖的過程中會不間斷地對一個標誌位進行讀寫，並且該種類比不允許亂序執行，因為這會破壞其機制。

更為常見的是使用作業系統提供的互鎖庫，這種庫通常設計為在有硬體支援時使用

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [11]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [全球資訊網] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結: [網路] /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [搜尋引擎] /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [robots.txt] /wiki/Robots.txt
外部連結: [網站] /wiki/%E7%BD%91%E7%AB%99
外部連結: [超連結] /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結: [HTML] /wiki/HTML
外部連結: [網頁] /wiki/%E7%B6%B2%E9%A0%81
外部連結: [網際網路] /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結: [伺服器] /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結: [超文字傳輸協定] /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [22]:
import os

def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        dir_name = 'WikiArticle'
        
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        
        filename = '{}/{}.txt'.format(dir_name, key_word)
        with open(filename, 'w', encoding='utf-8') as f:
            for paragraph in content:
                f.write(paragraph.get_text())
        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string['href']
                    a_keyword = link_string.get_text()
                    external_link_dict[a_link] = a_keyword                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [ ]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E4%BA%92%E6%96%A5%E9%8E%96 (互斥鎖)
遞迴層[1] - /wiki/%E5%A4%9A%E7%BA%BF%E7%A8%8B (多執行緒)
遞迴層[2] - /wiki/%E8%BD%AF%E4%BB%B6 (軟體)
遞迴層[2] - /wiki/%E7%A1%AC%E4%BB%B6 (硬體)
遞迴層[2] - /wiki/%E7%BA%BF%E7%A8%8B (執行緒)
遞迴層[2] - /wiki/%E5%AF%B9%E7%A7%B0%E5%A4%9A%E5%A4%84%E7%90%86%E6%9C%BA (對稱多處理機)
遞迴層[2] - /wiki/%E5%A4%9A%E6%A0%B8%E5%BF%83 (多核心)
遞迴層[2] - /wiki/%E8%8A%AF%E7%89%87%E7%BA%A7%E5%A4%9A%E5%A4%84%E7%90%86 (晶片級多處理)
遞迴層[2] - /wiki/%E5%90%8C%E6%97%B6%E5%A4%9A%E7%BA%BF%E7%A8%8B (同時多執行緒)
遞迴層[2] - /wiki/%E5%A4%84%E7%90%86%E5%99%A8 (處理器)
遞迴層[2] - /wiki/%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F (作業系統)
遞迴層[2] - /wiki/%E5%BE%AE%E8%BD%AF (微軟)
遞迴層[2] - /wiki/Linux (Linux)
遞迴層[2] - /wiki/DOS (DOS)
遞迴層[2] - /wiki/%E5%A4%9A%E5%9F%B7%E8%A1%8C%E7%B7%92_(%E9%9B%BB%E8%85%A6%E7%A1%AC%E9%AB%94) (硬體多執行緒技術)
遞迴層[2] - /wiki/%E8%8A%AF%E7%89%87%E7%BA%A7%E5%88%AB%E7%9A%84%E5%A4%9A%E5%A4%84%E7%90%86 (CMP)
遞迴層[2] - /wiki/Intel_Core (Core)
遞迴層[2] - /wiki/%E5%8D%87%E9%98%B3 (Sun)
遞迴層[2] - /wiki/UltraSPARC_T1 (UltraS